In [11]:
import sys
import os
import time

sys.path.append(os.path.abspath(".."))


from dask import dataframe as dd
from cluster_config.slurm_cluster import *

In [12]:
client = get_slurm_cluster()

2025-06-11 15:17:28,837 - tornado.application - ERROR - Exception in callback functools.partial(<bound method IOLoop._discard_future_result of <tornado.platform.asyncio.AsyncIOMainLoop object at 0x000001CBD24ACB00>>, <Task finished name='Task-8145' coro=<SpecCluster._correct_state_internal() done, defined at c:\Users\natal\PycharmProjects\Large-scale-dataset-processing-using-Dask\.conda\Lib\site-packages\distributed\deploy\spec.py:352> exception=NotImplementedError()>)
Traceback (most recent call last):
  File "C:\Users\natal\AppData\Roaming\Python\Python312\site-packages\tornado\ioloop.py", line 758, in _run_callback
    ret = callback()
          ^^^^^^^^^^
  File "C:\Users\natal\AppData\Roaming\Python\Python312\site-packages\tornado\ioloop.py", line 782, in _discard_future_result
    future.result()
  File "c:\Users\natal\PycharmProjects\Large-scale-dataset-processing-using-Dask\.conda\Lib\site-packages\distributed\deploy\spec.py", line 396, in _correct_state_internal
    await asyn

Dashboard: http://192.168.1.113:8787/status


In [8]:
time_start = time.perf_counter()
df = dd.read_parquet("../data/yellow_tripdata_2023-*.parquet", blocksize="16MB")
time_elapsed = time.perf_counter() - time_start
print(f'Time to read data: {time_elapsed:.2f} seconds')

Time to read data: 0.03 seconds


In [9]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.SLURMCluster
Dashboard: http://192.168.1.113:8787/status,
Dashboard: http://192.168.1.113:8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://192.168.1.113:50525,Workers: 0
Dashboard: http://192.168.1.113:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [10]:
df['month'] = df['tpep_pickup_datetime'].dt.month
df['year'] = df['tpep_pickup_datetime'].dt.year
df['trip_distance_km'] = df['trip_distance'] * 1.60934

# Grouping
time_start = time.perf_counter()
courses_per_month_and_year = df.groupby(['month', 'year']).size().compute()
time_elapsed = time.perf_counter() - time_start
print(f'Time to group data: {time_elapsed:.2f} seconds')

KeyboardInterrupt: 